# Urban Water Model Simulation for Fehraltorf

This notebook runs the urban water model simulation for Fehraltorf and visualizes the results.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

from ipywidgets import Button, Output
import ipywidgets as widgets
import plotly.graph_objects as go

import holoviews as hv
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

from duwcm import run
from duwcm.read_data import read_data
from duwcm.functions import load_config
from duwcm.postprocess import extract_local_results
from duwcm.data_structures import UrbanWaterData

from duwcm.viz import (
    plot_aggregated_results,
    create_map_base,
    create_dynamic_map,
    create_flow_visualization,
    create_reuse_visualization
)

## Configuration

Set up the configuration for the simulation.

In [ ]:
config_path = '.'
config = load_config(config_path, env='default')

print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.output_directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

geo_file = Path(config.input_directory) / Path(config.files.geo_file)
background_file = Path(config.geodata_directory) / config.files.background_shapefile

_, _, _, _, _, flow_paths = read_data(config)

selected_cells = set()
selection_output = Output()
apply_button = widgets.Button(description='Apply Selection')

def create_selectable_map():
    gdf_geometry = gpd.read_file(geo_file)
    fig = create_map_base(geo_file, background_file, flow_paths)
    
    # Set proper customdata for selection
    fig.data[0].customdata = gdf_geometry['BlockID'].values
    fig.data[0].hovertemplate = "Cell ID: %{customdata}<extra></extra>"
    
    # Proper styling for selection state
    fig.data[0].selected = dict(marker=dict(opacity=1.0))
    fig.data[0].unselected = dict(marker=dict(opacity=0.3))
    
    fig.update_layout(
        clickmode='event+select',
        dragmode='select',
        showlegend=False,
        height=800,
        updatemenus=[]
    )
    return fig

selected_cells = set()
selection_output = Output()

def selection_fn(trace, points, selector):
    if points.point_inds:
        selected_cells.clear()  # Clear previous selection
        for i in points.point_inds:
            cell_id = int(trace.customdata[i])
            selected_cells.add(cell_id)

def apply_selection(b):
    with selection_output:
        selection_output.clear_output()
        if selected_cells:
            config.grid['selected_cells'] = sorted(list(selected_cells))
            print(f"Selected cells: {config.grid['selected_cells']}")
        else:
            if 'selected_cells' in config.grid:
                del config.grid['selected_cells']
            print("No cells selected - will use all cells")

def clear_selection(b):
    selected_cells.clear()
    if 'selected_cells' in config.grid:
        del config.grid['selected_cells']
    with selection_output:
        selection_output.clear_output()
        print("Selection cleared")

# Create buttons
apply_button = widgets.Button(description='Apply Selection')
clear_button = widgets.Button(description='Clear Selection')

apply_button.on_click(apply_selection)
clear_button.on_click(clear_selection)

# Create and display interface
base_map = create_selectable_map()
fig_widget = go.FigureWidget(base_map)
fig_widget.data[0].on_selection(selection_fn)

display(widgets.VBox([
    fig_widget,
    widgets.HBox([apply_button, clear_button]),
    selection_output
]))

## Run Simulation

Execute the urban water model simulation.

In [ ]:
results, forcing_data, flow_paths = run(config)

## Visualize Results

Create plots to visualize the simulation results.

In [ ]:
fig = plot_aggregated_results(results['aggregated'], forcing_data)
fig.show()

## Generate Interactive Maps

Create interactive map visualizations of the results.

In [ ]:
area_variables = [
    'evapotranspiration',
    'imported_water',
    'baseflow',
    'deep_seepage',
    'stormwater_runoff',
    'sewerage_discharge',
    'groundwater',
    'vadose_moisture'
]

gdf_geometry = gpd.read_file(geo_file)
local_results = extract_local_results(results)
# Filter results to selected cells if specified
if hasattr(config.grid, 'selected_cells'):
    local_results = local_results[local_results.index.get_level_values('cell').isin(config.grid.selected_cells)]

time_series_data = local_results[area_variables].unstack(level='cell')
time_series_data.attrs['units'] = local_results.attrs['units']

dynamic_map = create_dynamic_map(gdf_geometry, background_file, area_variables, time_series_data)
dynamic_map.show()

## Generate Alluvial diagrams

In [ ]:
sankey_fig = create_flow_visualization(results, viz_type='sankey')
sankey_fig.show()

In [ ]:
sankey_fig = create_reuse_visualization(results, viz_type='sankey')
sankey_fig.show()

## Generate Chord diagrams

In [ ]:
chord_fig = create_flow_visualization(results, viz_type='chord') 
chord_fig

In [ ]:
chord_fig = create_reuse_visualization(results, viz_type='chord') 
chord_fig